In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

kuka_output = pd.read_excel('KukaOutput-ICNAP.xlsx', engine='openpyxl')
kuka_output['EndDate'] = pd.to_datetime(kuka_output['EndDate'])
t0 = kuka_output['EndDate'].iloc[0]
kuka_output['EndDate min'] = (kuka_output['EndDate'] - t0).astype('timedelta64[m]')
programs = pd.read_excel('OvenPrograms-ICNAP.xlsx', engine='openpyxl')

In [70]:
class Oven:
    def __init__(self, index, finish_time=0, temp=0, rotor_list=[]):
        self.index = index
        self.finish_time = finish_time
        self.temp = temp
        self.rotor_list = rotor_list
    
    def get_time_per_product(self, time_available, duration, l):
        if time_available > self.finish_time:
            return time_available + (duration/l)
        return self.finish_time + (duration/l)

    def get_enter_time(self, time_available):
        if time_available > self.finish_time:
            return time_available
        return self.finish_time
    
    def dif_temperature(self, target_temp):
        return target_temp-self.temp
    
    def add_rotor(self, time_available, duration, cold_temp, index):
        self.temp = cold_temp
        self.rotor_list.append(index)
        if time_available > self.finish_time:
            self.finish_time = time_available + duration
            
        else:
            self.finish_time += duration
        
class Node:
    def __init__(self, path, remaining_df, ovens, node_dict):
        self.path = path
        self.remaining_df = remaining_df
        self.ovens = ovens
        self.node_dict = node_dict
        
    def get_neighboors(self):
        if self.remaining_df.shape[0] > 10:
            max_iter = 10
        else:
            max_iter = self.remaining_df.shape[0]
        
        ns = [self.get_neighboor(i) for i in range(max_iter)]
        return list(filter(lambda n: n['Lambda']<=max_lambda, ns))
            
    def get_next(self):
        neighboors = self.get_neighboors()
        best_cost = min([n['MinCost'] for n in neighboors])
        next_n_index = [n['MinCost'] for n in neighboors].index(min([n['MinCost'] for n in neighboors]))
        return neighboors[next_n_index]
    
    def get_neighboor(self, index):
        n = dict()
        n['Product'] = self.remaining_df['Product'].iloc[index]
        n['TimeAvailable'] = self.remaining_df['EndDate min'].iloc[index]
        n['Duration'] = programs[programs['product type']==n['Product']]['program duration (min)'].values[0] + 60
        n['TargetTemp'] = programs[programs['product type']==n['Product']]['target temp (C)'].values[0]
        n['CoolTemp'] = programs[programs['product type']==n['Product']]['cooling temp  (C)'].values[0]
        df_index = self.remaining_df.iloc[:index+1]
        n['Lambda'] = df_index[df_index['Product']==n['Product']].shape[0]
        n['ProductIndexes'] = list(df_index[df_index['Product']==n['Product']].index)
        n['ProductTimes'] = [oven.get_time_per_product(n['TimeAvailable'], n['Duration'], n['Lambda']) for oven in self.ovens]
        n['ProductDifTemps'] = [oven.dif_temperature(n['TargetTemp'])/n['Lambda'] for oven in self.ovens]
        n['Costs'] = [self.calculate_cost(n['ProductTimes'][i], n['ProductDifTemps'][i]) for i in range(len(n['ProductTimes']))]
        n['MinCost'] = min(n['Costs'])
        n['OvenIndex'] = n['Costs'].index(n['MinCost'])
        n['EnterTime'] = self.ovens[n['OvenIndex']].get_enter_time(n['TimeAvailable'])
        n['DifTemperature'] = self.ovens[n['OvenIndex']].get_enter_time(n['TargetTemp'])
        return n
    
    def calculate_cost(self, temp, time):
        percentage_temp = temp*0.9/291
        percentage_time = time*0.1/920
        return percentage_temp+percentage_time

In [71]:
oven_list = [Oven(i, finish_time=0, temp=0, rotor_list=[]) for i in range(6)]
path = []
df_remaining = kuka_output.copy()
current_node = Node(path, df_remaining, oven_list, {})
max_lambda = 3

while df_remaining.shape[0]>0:
    next_node_dict = current_node.get_next()
    oven_list[next_node_dict['OvenIndex']].add_rotor(next_node_dict['TimeAvailable'], next_node_dict['Duration'],
                                                      next_node_dict['CoolTemp'], next_node_dict['ProductIndexes'])
    df_remaining.drop(index=next_node_dict['ProductIndexes'], inplace=True)
    path.append(current_node)
    current_node = Node(path, df_remaining, oven_list, next_node_dict)
    if df_remaining.shape[0] == 0:
        path.append(current_node)

node_dicts = [i.node_dict for i in path][1:]
nodes_df = pd.DataFrame(node_dicts)
nodes_df['JobStart'] = t0 + pd.to_timedelta(nodes_df['EnterTime'], unit='m')
nodes_df['JobFinish'] = t0 + pd.to_timedelta(nodes_df[['EnterTime', 'Duration']].sum(axis=1), unit='m')
nodes_df.head()

,Product,TimeAvailable,Duration,TargetTemp,CoolTemp,Lambda,ProductIndexes,ProductTimes,ProductDifTemps,Costs,MinCost,OvenIndex,EnterTime,DifTemperature,JobStart,JobFinish
0,Rotor A3,1.0,450,319.55085,42.60678,2,"[0, 1]","[226.0, 226.0, 226.0, 226.0, 226.0, 226.0]","[159.775425, 159.775425, 159.775425, 159.77542...","[0.7163359661866877, 0.7163359661866877, 0.716...",0.716336,0,1.0,319.55085,2022-02-01 01:02:47,2022-02-01 08:32:47
1,Rotor A3,338.0,450,319.55085,42.60678,2,"[4, 5]","[676.0, 563.0, 563.0, 563.0, 563.0, 563.0]","[138.472035, 159.775425, 159.775425, 159.77542...","[2.10577295763671, 1.758604007423801, 1.758604...",1.758604,1,338.0,319.55085,2022-02-01 06:39:47,2022-02-01 14:09:47
2,Rotor C1,343.0,240,333.75311,42.60678,1,[6],"[691.0, 1028.0, 583.0, 583.0, 583.0, 583.0]","[291.14633, 291.14633, 333.75311, 333.75311, 3...","[2.168759742279247, 3.2110277835163608, 1.8393...",1.839370,2,343.0,333.75311,2022-02-01 06:44:47,2022-02-01 10:44:47
3,Rotor A2,365.0,450,319.55085,42.60678,2,"[3, 7]","[676.0, 1013.0, 808.0, 590.0, 590.0, 590.0]","[138.472035, 138.472035, 138.472035, 159.77542...","[2.10577295763671, 3.148040998873823, 2.514020...",1.842109,3,365.0,319.55085,2022-02-01 07:06:47,2022-02-01 14:36:47
4,Rotor A4,127.0,460,319.55085,42.60678,1,[2],"[911.0, 1248.0, 1043.0, 1275.0, 587.0, 587.0]","[276.94407, 276.94407, 276.94407, 276.94407, 3...","[2.847628389500224, 3.8898964307373376, 3.2558...",1.850198,4,127.0,319.55085,2022-02-01 03:08:47,2022-02-01 10:48:47


In [72]:
fig = px.timeline(nodes_df, x_start='JobStart', x_end='JobFinish', y="OvenIndex", color="Product", text="ProductIndexes")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()